<h1><center>cs1001.py , Tel Aviv University, Spring 2018</center></h1>
<img src="http://www.pngall.com/wp-content/uploads/2016/05/Python-Logo-PNG-Image-180x180.png" width=50/>

# Recitation 6

Recursion recursion recursion!

We saw two recursive functions for finding the maximal element in a list.
We discussed quicksort.
Then we wrote two recursive functions: binom and change.
We also discussed memoization and demonstrated it using our recursive implementations for binom and change.


#### Takeaways:
- The recursion tree helps in bounding the recursion depth and time complexity. Each tree node represents a call to the recursive function. We write the relevant size of the input inside the tree node, and for each node we also keep the total amount of work done for this input, not including the time spent on the recursive call.
- It is important to find out the recursion depth (as an O(.) bound). Note that python has a limit on this value.
- The time complexity of a recursive function is the total amount of work in all the tree nodes.
- Memoization is mainly technical. Remember the main steps of defining an envelope function, deciding what keys you use to describe the input, 
and finally changing your recursive implementation so that it will search for the key in the dictionary before making the recursive calls, and save the key:value pair after obtaining the value for a certain input. 
- The keys of the dictionary should be chosen to represent the current input to the function in a one-to-one fashion.
- When analyzing the time complexity of a recursive function with memoization, consider the recursion tree and remember that a node that has already been computed will not have a subtree.

#### Code for printing several outputs in one cell (not part of the recitation):

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Max1 
The maximum is the maximal value between lst[0] and the result of recursively finding the max in lst[1:].

Let $n$ denote the size of lst.

Recursion depth: $O(n)$

Time complexity: $O(n^2)$

In [4]:
def max1(L): 
    if len(L) == 1:
        return L[0]
    return max(max1(L[1:]), L[0])

## Max2
The maximum is the maximal value between the result of recursively finding the max in lst[:n//2] and the result of recursively finding the max in lst[n//2:], where $n$ denotes the size of lst.

Recursion depth: $O(\log{n})$

Time complexity: $O(n\log{n})$

In [5]:
def max2(L): 
    if len(L) == 1:
        return L[0]
    l = max2(L[:len(L)//2])
    r = max2(L[len(L)//2:])
    return max(l,r)

### comparison in running times

In [6]:
import time
import random
import sys
sys.setrecursionlimit(5000)


for f in [max1, max2]:
    print(f.__name__)
    for n in [1000,2000, 4000]:
        L = [i for i in range(n)]
        random.shuffle(L)
        tic =time.clock()
        for x in range(10):
            f(L)
        toc =time.clock()
        print("n=",n,": ",toc-tic)

max1
n= 1000 :  0.1126931254485437
n= 2000 :  0.342663700192405
n= 4000 :  1.9245601804517452
max2
n= 1000 :  0.012483430035867826
n= 2000 :  0.02438024978702824
n= 4000 :  0.048809054799506146


## Quick Sort

#### Random:

In [ ]:
def quicksort(lst):
    """ quick sort of lst """
    if len(lst) <= 1: 
        return lst
    else:
        pivot = random.choice(lst)         # select a random element from list
        smaller = [elem for elem in lst if elem < pivot] 
        equal = [elem for elem in lst if elem == pivot]      
        greater = [elem for elem in lst if elem > pivot]
        return quicksort(smaller) + equal + quicksort(greater) #two recursive calls

#### Deterministic:

In [ ]:
def det_quicksort(lst):
    """ sort using deterministic pivot selection """
    if len(lst) <= 1: 
        return lst
    else:
        pivot = lst[0]      # select first element from list
        smaller = [elem 
                   elem in lst if elem < pivot] 
        equal = [elem for elem in lst if elem == pivot]      
        greater = [elem for elem in lst if elem > pivot]
        return det_quicksort(smaller) + equal + det_quicksort(greater) #two recursive calls

## Binom

The number of sets of size $k$ selected from a set of $n$ elements (with no repetitions)
Recursive formula (Pascal):
$\binom{n}{k} = \binom{n-1}{k} + \binom{n-1}{k-1}$
where the stopping criterion is $\binom{n}{0} = \binom{n}{n} = 1$

The time complexity of binom is exponential in $n$ (worst case behaviour is when $k=\frac{n}{2}$)

In [12]:
def binom(n,k):
    if n < 0 or k < 0 or n < k:
        return 0
    elif (k==0 or n==k):
        return 1
    return binom(n-1,k-1) + binom(n-1,k)

binom(4,2)

6

#### Printing the recursive calls using tracing:

In [10]:
def binom_trace(n,k):
    result = binom_trace(n,k)
    return result

def binom_trace(n,k,indent=1):
    #indent = how much to indent the printouts
    if (k<0 or n<0 or n<k): # safety checks
        return 0
    elif (k==0 or k==n): # halting conditions
        print(">>"*indent + "({},{})".format(n,k))
        print("<<"*indent + "({},{})".format(n,k))
        return 1
    print(">>"*indent + "({},{})".format(n,k))
    indent+=1
    val = binom_trace(n-1,k,indent) + binom_trace(n-1,k-1,indent)
    indent-=1
    print("<<"*indent + "({},{})".format(n,k))
    return val

binom_trace(4,2)

>>(4,2)
>>>>(3,2)
>>>>>>(2,2)
<<<<<<(2,2)
>>>>>>(2,1)
>>>>>>>>(1,1)
<<<<<<<<(1,1)
>>>>>>>>(1,0)
<<<<<<<<(1,0)
<<<<<<(2,1)
<<<<(3,2)
>>>>(3,1)
>>>>>>(2,1)
>>>>>>>>(1,1)
<<<<<<<<(1,1)
>>>>>>>>(1,0)
<<<<<<<<(1,0)
<<<<<<(2,1)
>>>>>>(2,0)
<<<<<<(2,0)
<<<<(3,1)
<<(4,2)


6

In [13]:
binom(50,25)

KeyboardInterrupt: 

Now with memoization:

In [14]:
def binom_fast(n,k):
    d = {}
    return binom_mem(n,k,d)

def binom_mem(n,k,mem):
    if n < 0 or k < 0 or n < k:
        return 0
    elif (k==0 or n==k):
        return 1
    if (n,k) not in mem:
        mem[(n,k)] = binom_mem(n-1,k, mem) + \
                    binom_mem(n-1,k-1, mem)
    return mem[(n,k)]

binom_fast(4,2)
binom_fast(50,25)

6

126410606437752

And now with the content of the dictionary: 

In [3]:
def binom_mem(n,k,mem):
    if n < 0 or k < 0 or n < k:
        return 0
    elif (k==0 or n==k):
        return 1
    
    if (n,k) not in mem:
        mem[(n,k)] = binom_mem(n-1,k, mem) + \
                    binom_mem(n-1,k-1, mem)
        print (n,k, mem)
    return mem[(n,k)]

binom_mem(4,2,{})

2 1 {(2, 1): 2}
3 2 {(2, 1): 2, (3, 2): 3}
3 1 {(2, 1): 2, (3, 2): 3, (3, 1): 3}
4 2 {(2, 1): 2, (3, 2): 3, (3, 1): 3, (4, 2): 6}


6

Printing the recursive calls, with memoization:

In [5]:
def binom_fast_trace(n,k):
    mem = dict()
    result = binom_mem_trace(n,k,mem)
    return result

def binom_mem_trace(n,k,mem,indent=1):
    #indent = how much to indent the printouts
    if (k<0 or n<0 or n<k): # safety checks
        return 0
    elif (k==0 or k==n): # halting conditions
        print(">>"*indent + "({},{})".format(n,k))
        print("<<"*indent + "({},{})".format(n,k))
        return 1
    print(">>"*indent + "({},{})".format(n,k))
    indent+=1
    if (n,k) not in mem:
        mem[(n,k)] = binom_mem_trace(n-1,k,mem,indent) + binom_mem_trace(n-1,k-1,mem,indent)
    indent-=1
    print("<<"*indent + "({},{})".format(n,k))
    return mem[(n,k)]


binom_fast_trace(4,2)

>>(4,2)
>>>>(3,2)
>>>>>>(2,2)
<<<<<<(2,2)
>>>>>>(2,1)
>>>>>>>>(1,1)
<<<<<<<<(1,1)
>>>>>>>>(1,0)
<<<<<<<<(1,0)
<<<<<<(2,1)
<<<<(3,2)
>>>>(3,1)
>>>>>>(2,1)
<<<<<<(2,1)
>>>>>>(2,0)
<<<<<<(2,0)
<<<<(3,1)
<<(4,2)


6

#### Analysis of binom_fast(n,k):

Time complexity = number of different visited cells \* number of visits per cell \* time per visit (not including recursive calls)


<img src="binom_proof.PNG">

## Change problem

A bus driver needs to give an exact change and she has coins of limited types. She has infinite coins of each type.
Given the amount of change ($amount$) and the coin types (the list $coins$), how many ways are there? 

solution:
The requested value (denoted as $W(amount, coins)$) is equal to the number of ways to give the change when using coins of type $coins[-1]$ at least once plus the number of ways to give the change without using coins of type $coins[-1]$.
$W(amount, coins) = W(amount-coins[-1], coins) + W(amount, coins[:-1])$

stopping crtiteria:
- If $amount == 0$ return 1
- If $amount <0$ or $coins==[]$ return 0

This function change() below has an exponential time complexity.

In [15]:
def change(amount, coins):
    if amount == 0:
        return 1
    elif amount < 0 or coins == []:
        return 0
    return change(amount, coins[:-1]) +\
        change(amount - coins[-1], coins)
    
change(5, [1,2,3])

5

The change() recursion tree for $amount=5, coins = [1,2,3]$:
<img src="rec6_change_tree.PNG">

Now with memoization:

In [16]:
def change_fast(amount, coins):
    d = {}
    return change_mem(amount, coins, d)

def change_mem(amount, coins, d):
    if amount == 0:
        return 1
    elif amount < 0 or coins == []:
        return 0
    #if (amount, tuple(coins)) not in d:
    if (amount, len(coins)) not in d:
        #d[(amount, tuple(coins))] = \
        d[(amount, len(coins))] = \
            change_mem(amount, coins[:-1], d) +\
            change_mem(amount - coins[-1], coins, d)
    #return d[(amount, tuple(coins))]
    return d[(amount, len(coins))]

change_fast(500, [1,3,2])
        

21084

## Count paths

We solved question 2(a) from the 2015 fall semester exam (Moed B):
<img src="cnt_path_question.png">

In [5]:
def cnt_paths(L):
    if all_zeros(L):
        return 1
    
    result = 0
    for i in range(len(L)):
        if L[i] != 0:
            L[i] -= 1
            result += cnt_paths(L)
            L[i] += 1
    return result

def all_zeros(L):
    for i in L:
        if i != 0:
            return False
    return True

Now with memoization:

In [8]:
def cnt_paths_fast(L):
    d = {}
    return cnt_paths_mem(L,d)

def cnt_paths_mem(L,d):
    if all_zeros(L):
        return 1
    if tuple(L) not in d:
        result = 0
        for i in range(len(L)):
            if L[i] != 0:
                L[i] -= 1
                result += cnt_paths_mem(L, d)
                L[i] += 1
        d[tuple(L)] = result
    return d[tuple(L)]

def all_zeros(L):
    for i in L:
        if i != 0:
            return False
    return True

cnt_paths_fast([1,2])
cnt_paths_fast([1,2,80, 4, 7,6])

3

7486376454823865391082482192000